In [1]:
import os
import glob
import json
from transformers import PreTrainedTokenizerFast

import torch
import math
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer




/home/arturo_hernandez/anaconda3/envs/phylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

tokenizer = PreTrainedTokenizerFast(tokenizer_file="../data/tokenizer/tokenizer.json")

In [3]:
len(tokenizer)

7931

In [4]:
processed_data_path='../data/processed/'
file_list= glob.glob(os.path.join(processed_data_path,'*.json'))
with open(file_list[1], "r") as load_file:
    data= json.load(load_file)

In [9]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1 , d_model) # if batch
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        #pe = torch.zeros(max_len, d_model)
        #pe[:, 0::2] = torch.sin(position * div_term)
        #pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [19]:
class language_model(nn.Module):
    #def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
    #             nlayers: int, dropout: float = 0.5):
    def __init__(self, ntoken: int, d_model: int,
                 nhead: int,
                 dropout: float = 0.5):
        super().__init__()
        self.embedding = nn.Embedding(ntoken, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)

        self.encoder_layer = TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        #Default: False (seq, batch, feature)
    def forward(self, toke_list):
        #toke_list shape (seq, batch)
        #x shape (seq, batch, d_model)
        x= self.embedding(toke_list)
        x= self.pos_encoder(x)
        x= self.encoder_layer(x, is_causal=True)
        return x

In [20]:
ntoken= len(tokenizer)
d_model= 128
nhead=4

model= language_model(ntoken, d_model, nhead)

In [21]:
input= torch.tensor(data['indices'], ).view(-1,1)
input.shape

torch.Size([2425, 1])

In [22]:
yp= model(input)

RuntimeError: Need attn_mask if specifying the is_causal hint. You may use the Transformer module method `generate_square_subsequent_mask` to create this mask.

In [37]:
len(input)

2425

In [18]:
yp.shape

torch.Size([2425, 1, 128])